In [1]:
%%bash
pip install -r ../requirements.txt

In [ ]:
import pandas as pd
import ast
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
import spacy
import os

In [3]:
famous_cuisines = [
    "Italian",
    "French",
    "Chinese",
    "Japanese",
    "Korean",
    "Indian",
    "Mexican",
    "Thai",
    "Spanish",
    "Greek",
]

In [ ]:
data  = pd.read_csv('../../data/RAW_recipes.csv')
data.set_index('id', inplace=True)
data["tags"] = data["tags"].apply(ast.literal_eval)
data

,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
id,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
31490,a bit different breakfast pizza,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
112140,all in the kitchen chili,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
59389,alouette potatoes,45,68585,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
44061,amish tomato ketchup for canning,190,41706,2002-10-25,"[weeknight, time-to-make, course, main-ingredi...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...
486161,zydeco soup,60,227978,2012-08-29,"[ham, 60-minutes-or-less, time-to-make, course...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
493372,zydeco spice mix,5,1500678,2013-01-09,"[15-minutes-or-less, time-to-make, course, pre...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
308080,zydeco ya ya deviled eggs,40,37779,2008-06-07,"[60-minutes-or-less, time-to-make, course, mai...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8


In [5]:
data = data.dropna(subset=['name'])
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the lemmatizer and stopwords list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
additional = {
    "minutes", "easiest", "ever", "aww", "i", "can", "t", "believe", "it", "s", "stole", "the", "idea", "from","mirj", "andrea", " s ", "andreas",
    "viestad", "andes", "andersen", "an", "ana", "amy", "2 ww points", "on demand", "anelia", "amazing",
    "ashley", "ashton", "amazing", "make", "house", "smell", "malcolm", "amazingly", "killer", "perfect",
    "addictive", "leave", "u", "licking", "ur", "finger", "clean", "th", "recipe", "special", "time", "favorite",
    "aunt", "jane", "soft", "and", "moist", "licking", "famous", "non fruitcake", "true", "later",
    "nonbeliever", "believer", "comfort", "ultimate", "lover", "love", "easy", "ugly", "cc", "uncle", "bill", "tyler",
    "unbelievably", "unbelievable", "healthy", "fat", "free", "un", "melt", "mouth", "ummmmm", "umm", "ummmy", "nummy", "ummmm", "unattended",
    "unbaked", "ultra", "ultimately", "yummy", "rich", "quick", "rachael", "ray", "fail", "party", "florence",
    "fast", "light", "low", "carb", "snack", "wedding", "anniversary", "anne", "marie", "annemarie", "annette", "funicello", "syms",
    "byrn", "mike", "willan", "summer", "autumn", "winter", "spring", "burrel", "anna", "tres", "sweet", "uber",
    "homemade", "ann","best","j", "anite", "anitas", "anman", "angie", "angry", "simple", "difficult", "andy", "andrew", "ancient", "still", "another", "best", "go",
    "grant", "grandma", "amusement", "park", "instruction", "kitchen", "test", "ww", "almost", "empty", "dressing", "instant", "like", "le", "virtually",
    "home", "made", "guilt", "guilty", "delicious", "parfait", "forgotten", "forget", "forevermama", "diet", "can", "real", "former",
    "miss", "fabulous", "forever", "authentic", "fortnum", "mason", "kid", "foolproof", "football", "season", "diabetic",
    "two", "small", "one", "three", "four", "five", "thanksgiving", "dream", "foothill", "paula", "deen", "food", "processor", "safari", "processor",
    "traditional", "forbidden", "flavorful", "grandmag", "grandmama", "grandmaman", "grandma", "grandmom", "lena", "alicia", "alisa", "alice", "ali", "bit", "different",
    "eat", "family", "global", "gourmet", "yam", "yam", "emotional", "balance", "tonight", "feel", "cooking", "got", "birthday", "air", "way", "mr", "never", "weep", "half",
    "anything", "pour", "put", "fork", "say", "stove", "top", "thought", "prize", "winning", "add", "ad", "good", "better", "da", "style", "even", "bran", "fake", "fire", "beautiful"
    "l", "game", "day", "hate", "world", "minute", "type", "starbucks", "biggest", "dressed", "summertime", "elmer", "johnny", "depp", "c", "p", "h", "clove", "er", "star", "week",
    "affair", "elegant", "student", "z", "whole", "lotta", "w", "z", "b", "aaron", "craze", "a", "abc", "absolute", "absolut", "absolutely", "perfection", "delightful", "lazy", "morning",
    "abuelo", "abuelito", "abuelita", "abuela", "acadia", "accidental", "adam", "little", "interest", "addicting", "addie", "adele", "adelaide", "adi", "adie", "adriana",
    "adult", "affordable", "alison", "holst", "purpose", "allegheny", "allegedly", "original", "allergic", "ex", "allergy", "allergen", "allen", "poorman", "backyard",
    "alton", "brown", "whatever", "anthony", "anytime", "april", "fool", "ya", "fooled", "sandra", "lee", "edna", "emma", "emy", "evy", "eva", 'evelyn', "fannie", "fanny", "flo", "gladys", "helen", "grace", "ira", "irma",
    "isse", "jean", "janet", "jenny", "juju", "judy", "kathy", "kathi", "kellie", "kelly", "laura", "lee", "kay", "kathleen", "laura", "lee", "lesley", "lil", "linda", "liz", "lois", "louisse",
    "mag", 'martguerite', "margie", "marge", "maggie", "martha", "marylin", "marion", "mary", "marthy", "melody", "michel", "meda", "millie", "muriel", "myrna", "nelda", "nancy", "paulie", "phillis", "rae", "rebecca",
    "rose", "sadie", "sarah", "sara", "sue", "susan", "teresa", "theresa", "auntie", "em", "barbara", "barb", "irene", "lolo", "lori", "lu", "maebelle",
    "aunty", "aussie", "aurora", "austin", "l", "q"
    
    }
stop_words.update(additional) 

# Function to clean recipe names
def clean_recipe_names(recipes):
    cleaned_recipes = []
    
    for recipe in recipes:
        recipe = recipe.lower()
        recipe = re.sub(r'[^a-z\s]', '', recipe)
        
        recipe_words = recipe.split()
        
        # Lemmatize first
        recipe_words = [lemmatizer.lemmatize(word) for word in recipe_words]
        
        # Then remove stopwords
        recipe_words = [word for word in recipe_words if word not in stop_words]
        
        cleaned_recipe = " ".join(recipe_words)
        cleaned_recipes.append(cleaned_recipe)
    
    return cleaned_recipes

# Apply the cleaning function to the 'recipe_name' column
data['cleaned_name'] = clean_recipe_names(data['name'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maxboc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/maxboc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/var/folders/pg/1l4d75p90r565jztmcw4v8c00000gn/T/ipykernel_8465/3190058043.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cleaned_name'] = clean_recipe_names(data['name'])


In [6]:
TAG2CLASS = {
    # North America – United States
    "american": ["North America – United States"],
    "north-american": ["North America – United States"],
    "northeastern-united-states": ["North America – United States"],
    "californian": ["North America – United States"],
    "native-american": ["North America – United States"],
    "pennsylvania-dutch": ["North America – United States"],
    "hawaiian": ["North America – United States"],

    # North America – Canada
    "canadian": ["North America – Canada"],
    "british-columbian": ["North America – Canada"],
    "quebec": ["North America – Canada"],

    # Central America & Caribbean
    "mexican": ["Mexican", "Central America & Caribbean"],
    "costa-rican": ["Central America & Caribbean"],
    "guatemalan": ["Central America & Caribbean"],
    "caribbean": ["Central America & Caribbean"],
    "cuban": ["Central America & Caribbean"],
    "puerto-rican": ["Central America & Caribbean"],
    "creole": ["Central America & Caribbean"],

    # South America
    "argentine": ["South America"],
    "brazilian": ["South America"],
    "peruvian": ["South America"],
    "chilean": ["South America"],
    "colombian": ["South America"],
    "venezuelan": ["South America"],
    "ecuadorean": ["South America"],

    # Europe – Western
    "french": ["French", "Europe – Western"],
    "english": ["Europe – Western"],
    "scottish": ["Europe – Western"],
    "irish": ["Europe – Western"],
    "welsh": ["Europe – Western"],
    "dutch": ["Europe – Western"],
    "belgian": ["Europe – Western"],
    "austrian": ["Europe – Western"],
    "german": ["Europe – Western"],
    "italian": ["Italian", "Europe – Western"],
    "portuguese": ["Europe – Western"],
    "spanish": ["Spanish", "Europe – Western"],
    "greek": ["Greek", "Europe – Western"],

    # Europe – Northern
    "swedish": ["Europe – Northern"],
    "norwegian": ["Europe – Northern"],
    "finnish": ["Europe – Northern"],
    "icelandic": ["Europe – Northern"],
    "danish": ["Europe – Northern"],

    # Europe – Eastern/Central
    "russian": ["Europe – Eastern"],
    "hungarian": ["Europe – Eastern"],
    "czech": ["Europe – Eastern"],
    "georgian": ["Europe – Eastern"],

    # Middle East & North Africa (MENA)
    "middle-eastern": ["Middle East & North Africa"],
    "turkish": ["Middle East & North Africa"],
    "lebanese": ["Middle East & North Africa"],
    "iranian-persian": ["Middle East & North Africa"],
    "iraqi": ["Middle East & North Africa"],
    "palestinian": ["Middle East & North Africa"],
    "saudi-arabian": ["Middle East & North Africa"],
    "egyptian": ["Middle East & North Africa"],
    "moroccan": ["Middle East & North Africa"],
    "libyan": ["Middle East & North Africa"],
    "algerian": ["Middle East & North Africa"],
    "tunisian": ["Middle East & North Africa"],

    # Asia – East
    "chinese": ["Chinese", "Asia – East"],
    "beijing": ["Chinese", "Asia – East"],
    "chinese-new-year": ["Chinese", "Asia – East"],
    "japanese": ["Japanese", "Asia – East"],
    "korean": ["Korean", "Asia – East"],
    "mongolian": ["Asia – East"],

    # Asia – Southeast
    "vietnamese": ["Asia – Southeast"],
    "indonesian": ["Asia – Southeast"],
    "malaysian": ["Asia – Southeast"],
    "cambodian": ["Asia – Southeast"],
    "laotian": ["Asia – Southeast"],
    "thai": ["Thai", "Asia – Southeast"],
    "polynesian": ["Asia – Southeast"],

    # Asia – South
    "pakistani": ["Asia – South"],
    "nepalese": ["Asia – South"],
    "indian": ["Indian", "Asia – South"],

    # Jewish Diaspora
    "jewish-ashkenazi": ["Jewish Diaspora", "Middle East & North Africa"],
    "jewish-sephardi": ["Jewish Diaspora", "Middle East & North Africa"],

    # Catch-all / generic
    "asian": ["Asia – General"],
    "european": ["Europe – General"],
}
PRIORITIZED_TAGS = [
    # Europe – Western
    "french", "english", "scottish", "irish", "welsh",
    "dutch", "belgian", "austrian", "german", "italian",
    # North America – United States
    "american", "north-american", "northeastern-united-states",
    "californian", "native-american", "pennsylvania-dutch",
    "hawaiian",
    # North America – Canada
    "canadian", "british-columbian", "quebec",
    # Central America & Caribbean
    "mexican", "costa-rican", "guatemalan",
    "caribbean", "cuban", "puerto-rican", "creole",
    # South America
    "argentine", "brazilian", "peruvian",
    "chilean", "colombian", "venezuelan", "ecuadorean",
    # Europe – Northern
    "swedish", "norwegian", "finnish", "icelandic", "danish",
    # Europe – Eastern/Central
    "russian", "hungarian", "czech", "georgian",
    # Europe – Southern
    "portuguese",
    # Middle East & North Africa (MENA)
    "middle-eastern", "turkish", "lebanese",
    "iranian-persian", "iraqi", "palestinian",
    "saudi-arabian", "egyptian", "moroccan", "libyan",
    # Sub-Saharan Africa
    "south-african", "ethiopian", "nigerian",
    "angolan", "sudanese",
    # Asia – East
    "chinese", "beijing", "chinese-new-year",
    "japanese", "korean", "mongolian",
    # Asia – Southeast
    "vietnamese", "indonesian", "malaysian",
    "cambodian", "laotian",
    "polynesian",
    # Asia – South
    "pakistani", "nepalese",
    # Jewish Diaspora
    "jewish-ashkenazi", "jewish-sephardi",
    # Finally, the generic catch-alls
    "asian", "european",
]

_tag_rank = {tag: idx for idx, tag in enumerate(PRIORITIZED_TAGS)}

def map_tags_to_cuisines(tags, cleaned_name):
    known = set()

    for t in tags:
        if t in TAG2CLASS:
            known.add(t)

    for tag in TAG2CLASS.keys():
        if tag.lower() in cleaned_name.lower():
            known.add(tag)

    if not known:
        return []
    
    all_cuisines = []
    for tag in known:
        all_cuisines.extend(TAG2CLASS[tag])

    unique_cuisines = []
    for cuisine in all_cuisines:
        if cuisine not in unique_cuisines:
            unique_cuisines.append(cuisine)

    def get_cuisine_priority(cuisine):
        cuisine_tags = []
        for tag in known:
            if cuisine in TAG2CLASS[tag]:
                cuisine_tags.append(tag)
        
        best_rank = min(_tag_rank.get(tag, float('inf')) for tag in cuisine_tags)
        return best_rank

    sorted_cuisines = sorted(unique_cuisines, key=get_cuisine_priority)
    
    return sorted_cuisines


data['cuisines'] = data.apply(lambda row: map_tags_to_cuisines(row['tags'], row['cleaned_name']), axis=1)

data = data[data['cuisines'].apply(len) > 0]

/var/folders/pg/1l4d75p90r565jztmcw4v8c00000gn/T/ipykernel_8465/1863225890.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cuisines'] = data.apply(lambda row: map_tags_to_cuisines(row['tags'], row['cleaned_name']), axis=1)


In [ ]:
data.loc[:, 'count_cuisines'] = data['cuisines'].apply(len)
aggreate = data.groupby('count_cuisines').size().reset_index(name='count')
display(aggreate)
data.drop(columns=['count_cuisines'], inplace=True)
data.to_csv('../../data/processed/recipes_with_cuisine.csv')

,count_cuisines,count
0,1,43472
1,2,20585
2,3,25696
3,4,2053
4,5,512
5,6,143
6,7,35
7,8,4
8,9,1
